In [1]:
!pip install allennlp
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.2/594.2 kB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.0 MB/s eta 0:00:00


In [3]:
#EJECUTAR DOS VECES
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import TreebankWordTokenizer, sent_tokenize
#from nltk.tokenize import sent_tokenize, RegexpTokenizer
from collections import Counter
import re
from nltk.corpus import stopwords
import pandas as pd
import json
import nltk
from allennlp.modules.elmo import Elmo, batch_to_ids
import torch
import math
from sklearn.model_selection import train_test_split
import contractions
from sklearn.feature_extraction.text import TfidfVectorizer



nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

from gensim.models.keyedvectors import KeyedVectors

#Cargamos un modelo pre-entrenado limitándolo a las 400K palabras más comunes. TODO: recuerda reemplazar el path por la localización del modelo en tu Drive.
word_vectors = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/NLP/Word2Vec/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=400000)

Mounted at /content/drive


Cargamos el dataset

In [15]:
dt = pd.read_csv('dataset_reducido.csv')

Eliminamos la columna features ya que la mayoria de filas estan vacias. Tambien laguange_ft y languace_cld3 que son iguales que la columna language

In [16]:
dt = dt.drop(columns=['features', 'language_ft', 'language_cld3'])

print(dt.head())


                             title  tag           artist  year  views  \
0                     Summers Over  pop  Greg MacPherson  1999     25   
1                          Castles  pop        bodyGaard  2020     96   
2  No Worries Tiëstos Big Room Mix  pop            Tisto  2017    340   
3          Revenge Is Sweeter Edit  pop    The Veronicas  2015    134   
4                        Right Now  pop   MasterClassNYC  2020     29   

                                              lyrics       id language  
0  Summer's over. Calming down. All of my long sl...  1405536       en  
1  [Chorus]\nSomewhere out there there’s castle f...  6742005       en  
2  [Verse 1]\nI found a journal that I kept when ...  3262944       en  
3  I saw it in the news\nYou told me they were wr...  1835973       en  
4  RIGHT NOW\n[Malcolm X Intro]\n\nVerse 1\n\nIt'...  5630026       en  


Imprimimos los generos que hay para saber cuantos outputs tendras el modelo que hagamos en un futuro

In [17]:
genre_counts = dt['tag'].value_counts().sort_values(ascending=False)

print(genre_counts)


tag
pop        7500
rap        7500
rock       7500
country    7500
Name: count, dtype: int64


### **TOKENIZACION**

Cargamos completamente la letra de una cancion para saber como es la estructura. Podemos ver que a las letras se le añaden comentarios entre corchetes.

In [18]:

index = 860
print("Letra de la canción en la fila", index)
print(dt.loc[index, 'lyrics'])


Letra de la canción en la fila 860
I can't feel good
These thoughts about my mind, they're starting to blow
I will not get high
Cause you're the only one, to lie

Speeding in the shadows
Working, when nobody knows
Just sit and compose
Screaming out slows...

I feel complexed
Stress up ahead
I can't feel better, than before
I feel complexed
In this deadhead
Will this ever get an end
Struggling out of the blend
Will there ever be a day
When you'll finally die away

I won't feel good
Blowing up thoughts, striking through the peace
I won't get high
Shaking overdrive, like a hurricane
Anger, pain, lies, hate
Love, darkness, jealousy, rage
Piss off, despair, satisfaction, tears
Hope, ignorance, shame, complex

I feel complexed
Felt so afraid
I won't care 'bout you, anymore
I feel complexed
In this bloodshed
Will this ever get an end
Struggling out of the blend
Will there ever be a day
When you'll finally fade away

You are my reason to try
You are my reason to die
You are my reason to lie
Yo

Esta funcion convierte todos los caracteres en minisculas, eliminar los comentarios entre corchetes y los caracteres que no sean letras comunes o numeros.

In [19]:
def preprocess_lyrics(lyrics):
    lyrics = lyrics.lower()
    lyrics = contractions.fix(lyrics)
    lyrics = re.sub(r'\[.*?\]', '', lyrics)
    lyrics = re.sub(r"[^a-zA-Z0-9\s\n]", '', lyrics)
    return lyrics.strip()

Comprobamos que la funcion funcione con una frase ejemplo

In [20]:
example_lyrics = "[Intro] I'm don't know how to love you. Oh, love me like you do."
cleaned_lyrics = preprocess_lyrics(example_lyrics)
print(cleaned_lyrics)

i am do not know how to love you oh love me like you do


In [21]:
dt['lyrics'] = dt['lyrics'].apply(preprocess_lyrics)

Imprimimos de nuevo una letra complleta para ver que es el formato que queremos

In [22]:
index = 860
print("Letra de la canción en la fila", index)
print(dt.loc[index, 'lyrics'])

Letra de la canción en la fila 860
i cannot feel good
these thoughts about my mind they are starting to blow
i will not get high
because you are the only one to lie

speeding in the shadows
working when nobody knows
just sit and compose
screaming out slows

i feel complexed
stress up ahead
i cannot feel better than before
i feel complexed
in this deadhead
will this ever get an end
struggling out of the blend
will there ever be a day
when you will finally die away

i will not feel good
blowing up thoughts striking through the peace
i will not get high
shaking overdrive like a hurricane
anger pain lies hate
love darkness jealousy rage
piss off despair satisfaction tears
hope ignorance shame complex

i feel complexed
felt so afraid
i will not care bout you anymore
i feel complexed
in this bloodshed
will this ever get an end
struggling out of the blend
will there ever be a day
when you will finally fade away

you are my reason to try
you are my reason to die
you are my reason to lie
you ar

Dividimos el dataset en train, validation y test

In [23]:
train_data, temp_data = train_test_split(dt, test_size=0.3, random_state=42)

validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

dt['set_type'] = 'train'
dt.loc[dt.index.isin(validation_data.index), 'set_type'] = 'validation'
dt.loc[dt.index.isin(test_data.index), 'set_type'] = 'test'

In [24]:
dt.rename(columns={'set_type': 'split'}, inplace=True)

In [25]:
dt.to_csv('canciones.csv', index=False)

**Una vez dividido el dataset para todo el analisis que hagamos a continuacion usaremos solo la parte del dataset del**

Funciones tanto para dividir por oraciones como por palabras. En la de por oraciones lo que hemos hecho es dividir cuando haya un salto de linea ya que no se usan los signos de puntuacion en las letras. Por otro lado para la funcion por tokens hemos dividido con el treebank tokenizer y eliminado las stoprwords

In [ ]:
stop_words = set(stopwords.words('english'))
tokenizer = TreebankWordTokenizer()

def split_into_sentences(lyrics):
    sections = lyrics.split('\n')
    sentences = []
    for section in sections:
        if section.strip():
            sentences.extend(sent_tokenize(section.strip()))
    return sentences

def tokenize_lyrics(lyrics):
    tokens = tokenizer.tokenize(lyrics)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

Aplicamos las funciones y comprobamos el resultado

In [ ]:
nltk.download('punkt_tab')
train_data['sentences'] = train_data['lyrics'].apply(split_into_sentences)
train_data['tokens_palabras'] = train_data['lyrics'].apply(tokenize_lyrics)

print(train_data[['title', 'sentences', 'tokens_palabras']])

Vemos en detalle todas las oraciones que nos genera de una cancion

In [ ]:
index = 860
print("Oraciones de la canción en el índice", index)
print(train_data.loc[index, 'sentences'])

Creamos el bag of words

In [ ]:
all_words = []

for tokens in train_data['tokens_palabras']:
    all_words.extend(tokens)

bag_of_words = Counter(all_words)
print(bag_of_words.most_common(10))

### **TF_IDF**

**TF**

In [ ]:
def compute_tf(tokens):
    token_count = Counter(tokens)
    total_tokens = len(tokens)
    tf_scores = {token: count / total_tokens for token, count in token_count.items()}
    return tf_scores

train_data['tf'] = train_data['tokens_palabras'].apply(lambda x: compute_tf(x))


all_tf_scores = []
for tf_dict in train_data['tf']:
    all_tf_scores.extend(tf_dict.items())

tf_top5 = pd.DataFrame(all_tf_scores, columns=['term', 'tf']).nlargest(10, 'tf')
tf_bottom5 = pd.DataFrame(all_tf_scores, columns=['term', 'tf']).nsmallest(10, 'tf')

print("Top 5 términos con mayor TF:\n", tf_top5)
print("\nTop 5 términos con menor TF:\n", tf_bottom5)

**IDF**

In [ ]:
N = len(train_data)

def calculate_document_frequencies(token_lists):
    df = Counter()
    for tokens in token_lists:
        unique_tokens = set(tokens)
        for token in unique_tokens:
            df[token] += 1
    return df

document_frequencies = calculate_document_frequencies(train_data['tokens_palabras'])

def calculate_idf(df, total_documents):
    idf = {word: math.log(total_documents / df[word]) for word in df}
    return idf

idf = calculate_idf(document_frequencies, N)

all_idf_scores = list(idf.items())

idf_df = pd.DataFrame(all_idf_scores, columns=['term', 'idf'])

idf_top5 = idf_df.nlargest(5, 'idf')
idf_bottom5 = idf_df.nsmallest(5, 'idf')

print("Top 5 términos con mayor IDF:\n", idf_top5)
print("\nTop 5 términos con menor IDF:\n", idf_bottom5)

**TF-IDF**

Una vez tenemos el tf y el idf podemos obtener el tf-idf

In [ ]:
def calculate_tf_idf(tf_dict, idf_dict):
    tf_idf = {word: tf_value * idf_dict.get(word, 0) for word, tf_value in tf_dict.items()}
    return tf_idf

train_data['tf_idf'] = train_data['tf'].apply(lambda tf: calculate_tf_idf(tf, idf))

print(train_data[['tf_idf']].head(10))


Generamos las 5 palabras con mayor y menor tf-idf

In [ ]:
def find_top_bottom_tfidf(tfidf_series, top_n=10):
    all_tfidf = pd.Series({word: score for tfidf in tfidf_series for word, score in tfidf.items()})
    top_tfidf = all_tfidf.nlargest(top_n)
    bottom_tfidf = all_tfidf.nsmallest(top_n)
    return top_tfidf, bottom_tfidf

top_tfidf, bottom_tfidf = find_top_bottom_tfidf(train_data['tf_idf'])

print("Top 5 palabras con mayor TF-IDF:")
print(top_tfidf)

print("\nTop 5 palabras con menor TF-IDF:")
print(bottom_tfidf)



In [ ]:

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(train_data['tokens_palabras'].apply(lambda x: ' '.join(x)))
words = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=words)
tfidf_df['tag'] = train_data['tag']
genre_tfidf_avg = tfidf_df.groupby('tag').mean()


In [ ]:
top_tfidf_by_genre = genre_tfidf_avg.apply(lambda x: x.nlargest(10).index.tolist(), axis=1)
print("Top términos por género o artista:\n", top_tfidf_by_genre)


### **NO CONTEXTUALES: Word2Vec**

In [ ]:
not_in_vocab_words = []

for word in all_words:
    if word not in word_vectors.key_to_index:
        not_in_vocab_words.append(word)

total_words = len(all_words)
not_in_vocab_count = len(not_in_vocab_words)
percentage_not_in_vocab = (not_in_vocab_count / total_words) * 100 if total_words > 0 else 0

print(f'Total de palabras en el dataset: {total_words}')
print(f'Palabras no representables con Word2Vec: {not_in_vocab_count}')
print(f'Porcentaje de palabras no representables: {percentage_not_in_vocab:.2f}%')


Aqui podemosver algunas de las palabras de nuestro dataset que no estan reprensentadas en el word2vec. En general lo que esperabamos palabras inventdas, mal escritas, nombres...

In [ ]:
print("Algunas palabras no representadas por Word2Vec:")
for word in not_in_vocab_words[:20]:
    print(word)

Al comprobar las palabras no reprensentadas aqui si que podemos ver que hay algunas palabras que no estan reconocidas por las contracciones. Sin embargo, hemos comprobado que si al limpiar las lyrics hubiesemos dejado la comilla el resultado era peor ya que el porcentaje de palabras no representadas era mayor asi que hemos decidido dejarlo de esta manera que es la que mejor resultado nos daba.

In [ ]:
not_in_vocab_counter = Counter(not_in_vocab_words)

print("\nPalabras no representadas que más se repiten:")
for word, count in not_in_vocab_counter.most_common(10):
    print(f"{word}: {count}")

Hacemos una comprobacion extra y vemos si las palabras que mas se repiten en nuestro corpues estan representadas en word2vec y podemos ver que estan todas representadas

In [ ]:
def check_word_in_word2vec(word_list, word_vectors):
    for word in word_list:
        if word in word_vectors.key_to_index:
            print(f"La palabra '{word}' está en el vocabulario de Word2Vec.")
        else:
            print(f"La palabra '{word}' NO está en el vocabulario de Word2Vec.")

words_to_check = ["i'm","didn't"]
most_common_words = [word for word, count in bag_of_words.most_common(20)]

check_word_in_word2vec(most_common_words, word_vectors)


### **CONTEXTUALES: ELMO**

In [ ]:
options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

In [ ]:
elmo = Elmo(options_file, weight_file, 1, dropout=0)

In [ ]:
def get_elmo_embeddings(sentences):
    character_ids = batch_to_ids(sentences)
    embeddings = elmo(character_ids)
    return embeddings['elmo_representations'][0]

Comprobamos con una pequeña muestra que elmo funcione ya que lo usaremos mas adelante con machine learning

In [ ]:
sample_songs = train_data['sentences'].sample(n=3).tolist()
elmo_embeddings = get_elmo_embeddings(sample_songs)
print(elmo_embeddings)